<a href="https://colab.research.google.com/github/maryKanata/D.S.-M.L.-Portfolio/blob/main/EnergyConsuption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import required libraries
import pandas as pd
import numpy as np

In [ ]:
#Load dataset as dataframe
from google.colab import files
uploaded = files.upload()

Saving KAG_energydata_complete.csv to KAG_energydata_complete.csv


In [ ]:
df = pd.read_csv('KAG_energydata_complete.csv')

In [ ]:
#Short Overview
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [ ]:
#Show dimensions of dataframe
df.shape

(19735, 35)

In [ ]:
#Information's Data Frame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [ ]:
# How many null values?
df.isnull().sum().sum()

0

In [ ]:
# How many dublicated values?
df.duplicated().sum()

0

In [ ]:
# Convert object to date-time
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# Round all columns to 2 decimal digits
df = df.round(2)

In [ ]:
# Extract datetime features
df['year'] = df['date'].dt.year.astype(int)
df['month'] = df['date'].dt.month.astype(int)
df['day'] = df['date'].dt.day.astype(int)
df['hour'] = df['date'].dt.hour.astype(int)
df['minute'] = df['date'].dt.minute.astype(int)
df['second'] = df['date'].dt.second.astype(int)

In [ ]:
# Defining the features and the target variable
X = df.drop(columns=['Appliances','date'], axis=0)
y = df['Appliances']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, 
                                                    random_state=43)

In [ ]:
# Standardize the features
scaler = StandardScaler()
train_scaled = scaler.fit_transform(X_train)
test_scaled = scaler.transform(X_test)

In [ ]:
# Convert the target variable to a NumPy array
y_train_array = y_train.values.reshape(-1, 1)
y_test_array = y_test.values.reshape(-1, 1)

In [ ]:
# Standardize the target variable
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train_array)
y_test_scaled = target_scaler.transform(y_test_array)

In [ ]:
# Create the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(len(X),)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

In [ ]:
# Assuming your input data is stored in train_scaled with shape (num_samples, 33)
num_features = train_scaled.shape[1]
train_scaled_reshaped = np.reshape(train_scaled, (train_scaled.shape[0], num_features))

# Update the input shape of the first layer in your model to match the number of features
model = Sequential()
model.add(Dense(64, input_shape=(num_features,)))

In [ ]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Train the model
model.fit(train_scaled, y_train_scaled , epochs=100, batch_size=100)

Epoch 1/100
158/158 [==============================] - 1s 2ms/step - loss: 1.1814
Epoch 2/100
158/158 [==============================] - 0s 2ms/step - loss: 0.9451
Epoch 3/100
158/158 [==============================] - 0s 2ms/step - loss: 0.9003
Epoch 4/100
158/158 [==============================] - 0s 2ms/step - loss: 0.8825
Epoch 5/100
158/158 [==============================] - 0s 2ms/step - loss: 0.8720
Epoch 6/100
158/158 [==============================] - 0s 2ms/step - loss: 0.8644
Epoch 7/100
158/158 [==============================] - 0s 2ms/step - loss: 0.8591
Epoch 8/100
158/158 [==============================] - 0s 2ms/step - loss: 0.8543
Epoch 9/100
158/158 [==============================] - 0s 2ms/step - loss: 0.8509
Epoch 10/100
158/158 [==============================] - 0s 2ms/step - loss: 0.8480
Epoch 11/100
158/158 [==============================] - 0s 2ms/step - loss: 0.8464
Epoch 12/100
158/158 [==============================] - 0s 3ms/step - loss: 0.8443
Epoch 13/100


In [ ]:
# Evaluate the model
mse = model.evaluate(test_scaled, y_test_scaled)
print("Mean Squared Error:", mse)

124/124 [==============================] - 0s 1ms/step - loss: 0.7989
Mean Squared Error: 0.7988712787628174


The performance of the model appears to be excellent based on MSE as a metric, since MSE is closely enough to zero, meaning that model's predictions are closer to the actual values.